# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load data from weather

vacation_df = pd.read_csv("clean_city_data.csv")


In [3]:
vacation_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port alfred,-33.59,26.89,64.31,86,0,9.62,ZA,1602113997
1,1,rikitea,-23.12,-134.97,73.92,79,100,7.47,PF,1602114260
2,2,guane,22.20,-84.09,80.42,78,69,9.42,CU,1602114260
3,3,cape town,-33.93,18.42,59.00,87,17,10.29,ZA,1602114260
4,4,mataura,-46.19,168.86,46.99,60,85,8.01,NZ,1602114260


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps

fig = gmaps.configure(api_key=g_key)

# Create variables for coordinates and humidity

locations = vacation_df[['Lat', 'Lng']]
weights = vacation_df['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70, Wind speed less than 10 mph, Zero cloudiness.

weather_conditions_df = vacation_df.loc[(vacation_df["Max Temp"] >70) & (vacation_df["Max Temp"] <80) & (vacation_df["Cloudiness"] ==0) & (vacation_df["Wind Speed"] <10)]
weather_conditions_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,56,ajdabiya,30.76,20.23,76.44,39,0,5.66,LY,1602114264
84,87,erzin,36.96,36.20,73.99,82,0,8.01,TR,1602114266
116,119,marzuq,14.40,46.47,72.36,52,0,0.96,YE,1602114268
320,331,rasulnagar,32.33,73.78,71.60,83,0,4.09,PK,1602114283
370,383,gat,31.61,34.76,72.00,52,0,0.78,IL,1602114226
382,395,dunyapur,29.80,71.72,75.20,60,0,9.17,PK,1602114288
394,407,kisanga,-7.44,37.70,70.95,75,0,4.05,TZ,1602114289
435,448,nizwa,22.93,57.53,72.32,38,0,0.40,OM,1602114292
442,456,itamaraju,-17.04,-39.53,72.39,86,0,5.99,BR,1602114292
449,464,caravelas,-17.71,-39.25,76.10,83,0,9.84,BR,1602114293


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df['Hotel Name'] = ""
hotel_df

<ipython-input-14-cb2d5672db59>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
54,56,ajdabiya,30.76,20.23,76.44,39,0,5.66,LY,1602114264,
84,87,erzin,36.96,36.20,73.99,82,0,8.01,TR,1602114266,
116,119,marzuq,14.40,46.47,72.36,52,0,0.96,YE,1602114268,
320,331,rasulnagar,32.33,73.78,71.60,83,0,4.09,PK,1602114283,
370,383,gat,31.61,34.76,72.00,52,0,0.78,IL,1602114226,
382,395,dunyapur,29.80,71.72,75.20,60,0,9.17,PK,1602114288,
394,407,kisanga,-7.44,37.70,70.95,75,0,4.05,TZ,1602114289,
435,448,nizwa,22.93,57.53,72.32,38,0,0.40,OM,1602114292,
442,456,itamaraju,-17.04,-39.53,72.39,86,0,5.99,BR,1602114292,
449,464,caravelas,-17.71,-39.25,76.10,83,0,9.84,BR,1602114293,
